# Turn Back Hoax

In [22]:
import requests
from bs4 import BeautifulSoup
import csv
import random

def extract_titles_and_urls(soup):
    # Find all elements with class 'entry-title mh-loop-title'
    return soup.find_all(class_='entry-title mh-loop-title')

def extract_title(entry):
    # Extract the title
    title_text = entry.text.strip()
    if '[SALAH]' in title_text:
        title_text = title_text.replace('[SALAH]', '').strip()
    return title_text

def extract_url(entry):
    # Find the <a> tag within this entry and extract the URL
    url_tag = entry.find('a')
    if url_tag:
        return url_tag.get('href')
    else:
        return ""
    
def extract_text(url):
    news_page = requests.get(url)
    news_soup = BeautifulSoup(news_page.content, 'html.parser')
    text_content = news_soup.find('div', class_='entry-content mh-clearfix')
    if text_content:
        return text_content.text.strip()
    else:
        return "News content not found for: " + url

train_csv_file = open('training_data2.csv', 'w', newline='', encoding='utf-8')
test_csv_file = open('Fake.csv', 'w', newline='', encoding='utf-8')

train_csv_writer = csv.writer(train_csv_file)
test_csv_writer = csv.writer(test_csv_file)

train_csv_writer.writerow(['Title', 'URL', 'Text', 'Label'])
test_csv_writer.writerow(['Title', 'URL', 'Text'])

num_pages = 50
for page_number in range(1, num_pages + 1):
    url = f"https://turnbackhoax.id/page/{page_number}"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    entries = extract_titles_and_urls(soup)
    random.shuffle(entries)

    # Split the data into training (80%) and testing (20%) datasets
    num_train = int(0.8 * len(entries))
    train_entries = entries[:num_train]
    test_entries = entries[num_train:]

    # Keywords
    keywords = ["anies", "prabowo", "presiden", "cawapres", "pemilu", "ganjar", "caslon", "gibran", "tps", "mahfud", "muhaimin", "cak imin", "kpu", "pilpres", "amin", "caslon"]  # Add more keywords as needed

    # training
    for entry in train_entries:
        title = extract_title(entry)
        url = extract_url(entry)
        keyword_found = False
        for keyword in keywords:
            if keyword in title.lower() and keyword in url.lower():
                keyword_found = True
                break
        if keyword_found:
            text = extract_text(url)
            train_csv_writer.writerow([title, url, text, 0]) 
        else:
            continue

    # test
    for entry in test_entries:
        title = extract_title(entry)
        url = extract_url(entry)
        keyword_found = False
        for keyword in keywords:
            if keyword in title.lower() and keyword in url.lower():
                keyword_found = True
                break
        if keyword_found:
            text = extract_text(url)
            test_csv_writer.writerow([title, url, text])
        else:
            continue

train_csv_file.close()
test_csv_file.close()
